# Imports

In [45]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import KFold,LeaveOneOut,LeavePOut,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import HalvingRandomSearchCV

In [2]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

<rpy2.rinterface_lib.sexp.NULLType object at 0x136e97f10> [0]

# Getting Dataset Over R from Python

In [3]:
pandas2ri.activate()
ro.r('''
        library("worldfootballR")
        laliga <- load_understat_league_shots(league = "La liga")
     ''')
laliga = pandas2ri.rpy2py(ro.r['laliga'])
laliga.drop('league', axis=1, inplace=True)

→ Data last updated 2024-05-30 18:34:46.012307882309 UTC


We are going to use La Liga dataset between 2020-01-01 and 2024-06-08.

In [4]:
laliga = laliga[(laliga['date'] > '2020-01-01') & (laliga['date'] < '2024-06-10')]

# Data Manipulations and Fixes

We have a problem about NaN values and duplicate features, we are going to fix these problems by manipulating the data.

In [5]:
warnings.filterwarnings('ignore')

def fixDataNaN(df):
    with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.py2rpy(df)
pairs = [['x','X'],['y','Y'],['x_g','xG'],['h_a','home_away'],['shot_type','shotType'],['last_action','lastAction']]

def camel_case_columns(df):
    def camel_case(column_name):
        parts = column_name.split('_')
        return str(parts[0] + ''.join(x.title() for x in parts[1:]))
    
    new_columns = []
    for column in df.columns:
        if '_' in column:
            new_columns.append(camel_case(column))
        else:
            new_columns.append(str(column))
    
    df.columns = new_columns
    return df

def fixMergeColumns(dataList, pairs):
    for targetData in dataList:
        for pair in pairs:
            if pair[0] in targetData.columns and pair[1] in targetData.columns:
                targetData['{}'.format(pair[1])].fillna(targetData['{}'.format(pair[0])], inplace=True)
                targetData.drop(columns=['{}'.format(pair[0])], inplace=True)
        targetData = camel_case_columns(targetData)
        fixDataNaN(targetData)

fixMergeColumns([laliga], pairs)

In [6]:
#  NA_ string contained count

total_count = laliga.isnull().sum()

print(total_count)

id                    0
minute                0
result                0
X                     0
Y                     0
xG                    0
player                0
playerId              0
situation             0
season                0
shotType              0
matchId               0
homeTeam              0
awayTeam              0
homeGoals             0
awayGoals             0
date                  0
playerAssisted    10653
lastAction            0
homeAway              0
dtype: int64


We are going to train our model to predict whether the position ends up to a goal or not so we need to convert our goal and not goal situations to binary tags.

In [7]:
replacement_dict = {
    'Goal': 'Goal',
    'BlockedShot': 'No Goal',
    'MissedShots': 'No Goal',
    'SavedShot': 'No Goal',
    'ShotOnPost': 'No Goal',
    'OwnGoal': 'No Goal'
}

# 'result' sütununu değiştirme
laliga['result'] = laliga['result'].map(replacement_dict)

# Imbalancedness

Our data has imbalancedness problem which might cause our model to learn features of majority target variables better than the minority target variables, it might cause inaccurate predictions.

In [8]:
print(laliga['result'].value_counts())

result
No Goal    35980
Goal        4211
Name: count, dtype: int64


We are going to use SMOTE method for oversampling to fix the gap between minority and majority target variables. This method uses clustering methods to create new observations based on original ones.

In [9]:
Y = laliga['result']
x = laliga.drop('result', axis=1)

X = pd.get_dummies(x)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

print('Original dataset shape %s' % Counter(Y))
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(X, Y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'No Goal': 35980, 'Goal': 4211})
Resampled dataset shape Counter({'Goal': 35980, 'No Goal': 35980})


# Model

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.25, random_state=42, stratify=y_res)

We found the best model to use is Random Forest after comparing Decision Tree, Logistic Regression,XGBoost and Random Forest.

In [44]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

### Train - Test Accuracy Comparison to Check Overfitting

Our model seems to have learned train set perfectly, we might have suspected of overfitting much more than the current situation if the test accuracies were bad but accuracy and balanced accuracy scores on train and test sets are very close to each other. Our model is good to go, but we are going to check if hyperparameter tuning could improve our performance.

In [ ]:
test_accuracy = accuracy_score(y_test, y_pred)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
test_confusion_matrix = confusion_matrix(y_test, y_pred)

train_accuracy = accuracy_score(y_train, model.predict(x_train))
train_balanced_accuracy = balanced_accuracy_score(y_train, model.predict(x_train))
train_confusion_matrix = confusion_matrix(y_train, model.predict(x_train))

In [41]:
print('Test Accuracy: {}\nTrain Accuracy: {}\n'.format(test_accuracy, train_accuracy))
print('Test Balanced Accuracy: {}\nTrain Balanced Accuracy: {}\n'.format(test_balanced_accuracy, train_balanced_accuracy))
print('Test Confusion Matrix: \n{}\n\nTrain Confusion Matrix: \n{}'.format(test_confusion_matrix, train_confusion_matrix))

Test Accuracy: 0.962757087270706
Train Accuracy: 1.0

Test Balanced Accuracy: 0.962757087270706
Train Balanced Accuracy: 1.0

Test Confusion Matrix: 
[[8614  381]
 [ 289 8706]]

Train Confusion Matrix: 
[[26985     0]
 [    0 26985]]


# Hyperparameter Tuning

We are going to use Halving Random Search method and 5-Fold Cross Validation together to find best hyperparameters during evaluating our iterations by balanced accuracy scores.

Halving Random Search method uses elimination system as its base idea, best of the two compaired hyperparameter sets rises above on the leaderboard and we get the best hyperparameters after the final round. Another pro of Halving Random Search is to be much faster than the Random Search.

In [49]:
# #hyperparameter tuning for RandomForestClassifier

n_estimators = [int(x) for x in np.linspace(start=50, stop=1000)]
max_features = ['sqrt', 'log2', None]
max_depth = [int(x) for x in np.linspace(2, 20)]
max_depth.append(None)
min_samples_split = [2, 10, 10]
min_samples_leaf = [1, 10, 10]
bootstrap = [True, False]

#Halving search
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start=50, stop=5000, num=10)],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [int(x) for x in np.linspace(2, 20)],
    'min_samples_split': [2, 10, 10],
    'min_samples_leaf': [1, 10],
    'bootstrap': [True, False]
}
halving = HalvingRandomSearchCV(model, param_grid, factor=3, resource='n_samples', max_resources=5000, random_state=42, verbose=3,scoring='balanced_accuracy', n_jobs=-1)
halving.fit(x_train, y_train)
print("Best Params:{}/nBest Balanced Accuracy:{}".format(halving.best_params_,halving.best_score_))

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 20
max_resources_: 5000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 250
n_resources: 20
Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV 2/5] END bootstrap=True, max_depth=7, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=2250;, score=(train=0.500, test=0.500) total time=   1.9s
[CV 4/5] END bootstrap=True, max_depth=7, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=2250;, score=(train=0.500, test=0.500) total time=   2.1s
[CV 3/5] END bootstrap=True, max_depth=7, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=2250;, score=(train=0.500, test=0.500) total time=   2.1s
[CV 1/5] END bootstrap=False, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=2, n_estimators=50;, score=(train=0.500, test=0.500) total time=   0.1s
[CV 2/5] END bootstrap=False, max_depth

/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV 3/5] END bootstrap=False, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=2, n_estimators=50;, score=(train=0.500, test=0.500) total time=   0.2s
[CV 4/5] END bootstrap=False, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=2, n_estimators=50;, score=(train=0.500, test=0.500) total time=   0.0s
[CV 5/5] END bootstrap=False, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=2, n_estimators=50;, score=(train=0.500, test=0.000) total time=   0.0s


/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV 2/5] END bootstrap=True, max_depth=17, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   3.5s
[CV 1/5] END bootstrap=True, max_depth=17, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   3.5s
[CV 3/5] END bootstrap=True, max_depth=17, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   3.6s
[CV 4/5] END bootstrap=True, max_depth=17, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   3.8s
[CV 2/5] END bootstrap=True, max_depth=13, max_features=sqrt, min_samples_leaf=10, min_samples_split=10, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   3.5s
[CV 5/5] END bootstrap=True, max_depth=17, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=33

...

[CV 1/5] END bootstrap=True, max_depth=13, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   5.7s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=3900;, score=(train=0.714, test=0.000) total time=   6.6s
[CV 1/5] END bootstrap=False, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1150;, score=(train=0.600, test=0.500) total time=   1.7s
[CV 2/5] END bootstrap=False, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1150;, score=(train=1.000, test=1.000) total time=   1.4s
[CV 4/5] END bootstrap=False, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1150;, score=(train=0.917, test=0.500) total time=   1.6s
[CV 3/5] END bootstrap=False, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1150;, s

/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV 5/5] END bootstrap=False, max_depth=3, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1150;, score=(train=0.929, test=0.750) total time=   1.7s
[CV 2/5] END bootstrap=True, max_depth=13, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   6.6s
[CV 3/5] END bootstrap=True, max_depth=13, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   5.9s


/Users/sezaiufukoral/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2466: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV 4/5] END bootstrap=True, max_depth=13, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=3350;, score=(train=0.500, test=0.500) total time=   5.7s
[CV 5/5] END bootstrap=True, max_depth=13, max_features=log2, min_samples_leaf=10, min_samples_split=2, n_estimators=3350;, score=(train=0.500, test=0.000) total time=   5.5s
----------
iter: 1
n_candidates: 84
n_resources: 60
Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV 1/5] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2250;, score=(train=1.000, test=0.722) total time=   3.8s
[CV 2/5] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2250;, score=(train=0.917, test=0.500) total time=   3.8s
[CV 3/5] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=2250;, score=(train=1.000, test=0.557) total time=   3.9s
[CV 2/5] END boot